In [1]:
import os
import gym
import numpy as np
import cv2
import torch
import difflib
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import os
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
# Adjust import based on your GenAI package structure

# Initialize the GenAI client with your API key
# client = Client(api_key="YOUR_API_KEY")
client = genai.Client(api_key="YOUR_KEY")

# Define your prompts
with open("list_of_prompts.txt", "r", encoding="utf-8") as file:
    prompts = [line.strip() for line in file if line.strip()]


output_base_dir = 'Dataset/imagen-generated'
os.makedirs(output_base_dir, exist_ok=True)

# Loop through prompts and generate images
# Loop through prompts and generate images
for prompt_idx, prompt in enumerate(prompts, start=1):
    folder_name = f"prompt_{prompt_idx}"
    folder_path = os.path.join(output_base_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # Save the prompt text in a file
    with open(os.path.join(folder_path, f"prompt_{prompt_idx}.txt"), "w") as file:
        file.write(prompt)

    # Generate 5 images for the current prompt
    response1 = client.models.generate_images(
        model="imagen-3.0-generate-002",
        prompt=prompt,
        config=types.GenerateImagesConfig(number_of_images=5)
    )

    # Generate 5 additional image for the current prompt
    response2 = client.models.generate_images(
        model="imagen-3.0-generate-002",
        prompt=prompt,
        config=types.GenerateImagesConfig(number_of_images=5)
    )

    # Combine generated images from both responses
    combined_generated_images = response1.generated_images + response2.generated_images

    # Save each generated image
    for image_num, generated_image in enumerate(combined_generated_images, start=1):
        # Extract image bytes and open the image
        image_bytes = generated_image.image.image_bytes
        image = Image.open(BytesIO(image_bytes))

        # Save the image file
        image_path = os.path.join(folder_path, f"prompt{prompt_idx}_image{image_num}.jpg")
        image.save(image_path)

    print(f"Saved 5 images for '{prompt}' in '{folder_path}'")